# sparql_handles

> SPARQL query execution with first-class result handles

In [ ]:
#| default_exp sparql_handles

## Overview

This module implements Stage 3 from the trajectory: SPARQL query execution with first-class result handles.

### Result Handle Pattern

Every SPARQL execution produces a `SPARQLResultHandle` with:
- `meta`: query, endpoint/local, timestamp, row count, columns
- `rows`: stored internally as list of dicts (SELECT) or Graph (CONSTRUCT/DESCRIBE)
- Bounded view operations: `res_head()`, `res_where()`, `res_group()`, `res_sample()`

### Progressive Disclosure

Result handles enable the root model to refine queries by inspecting metadata and small slices, not rerunning blind queries.

### Dataset Integration

SPARQL results can optionally be stored in dataset work graphs with full provenance tracking.

## Imports

In [ ]:
#| export
from sparqlx import SPARQLWrapper
from rdflib import Graph, URIRef, Literal
from dataclasses import dataclass, field
from datetime import datetime
from pathlib import Path
from functools import partial
from itertools import islice
import random
from typing import Optional

## SPARQLResultHandle

Unified wrapper for all SPARQL result types with metadata and bounded view operations.

In [ ]:
#| export
@dataclass
class SPARQLResultHandle:
    """Wrapper for SPARQL results with metadata and bounded view operations."""

    # Result data (never dumped wholesale to LLM)
    rows: list | Graph          # SELECT rows or CONSTRUCT/DESCRIBE graph
    result_type: str            # 'select' | 'ask' | 'construct' | 'describe'

    # Metadata
    query: str                  # Original SPARQL query
    endpoint: str               # Where executed (URL or 'local')
    timestamp: str = field(default_factory=lambda: datetime.utcnow().isoformat() + 'Z')

    # For SELECT results
    columns: list = None        # Column names
    total_rows: int = 0         # Total before limit (may be same as len(rows) if not truncated)

    # For Graph results (CONSTRUCT/DESCRIBE)
    triple_count: int = 0       # Number of triples actually stored in self.rows
    total_triples: int = 0      # Total before limit (may be same as triple_count if not truncated)

    def summary(self) -> str:
        """Bounded summary for LLM - reports actual stored count."""
        if self.result_type == 'select':
            stored = len(self.rows)
            if self.total_rows > stored:
                return f"SELECT: {stored} rows (of {self.total_rows} total), columns={self.columns}"
            return f"SELECT: {stored} rows, columns={self.columns}"
        elif self.result_type == 'ask':
            return f"ASK: {self.rows}"
        else:
            # For CONSTRUCT/DESCRIBE, report actual stored count
            if self.total_triples > self.triple_count:
                return f"{self.result_type.upper()}: {self.triple_count} triples (of {self.total_triples} total)"
            return f"{self.result_type.upper()}: {self.triple_count} triples"

    def __len__(self):
        if isinstance(self.rows, bool):
            return 1  # ASK result
        return len(self.rows) if hasattr(self.rows, '__len__') else 0

    def __iter__(self):
        if isinstance(self.rows, bool):
            return iter([self.rows])
        return iter(self.rows)
    
    def __repr__(self):
        return f"SPARQLResultHandle({self.summary()})"

Test SPARQLResultHandle with different result types:

In [ ]:
# Test SELECT result
select_handle = SPARQLResultHandle(
    rows=[{'s': 'http://ex.org/alice', 'age': '30'}],
    result_type='select',
    query='SELECT ?s ?age WHERE { ?s :age ?age }',
    endpoint='local',
    columns=['s', 'age'],
    total_rows=1
)
assert select_handle.summary() == "SELECT: 1 rows, columns=['s', 'age']"
assert len(select_handle) == 1
print(f"✓ SELECT handle: {select_handle}")

# Test SELECT with truncation
truncated_select = SPARQLResultHandle(
    rows=[{'s': 'http://ex.org/alice', 'age': '30'}],
    result_type='select',
    query='SELECT ?s ?age WHERE { ?s :age ?age }',
    endpoint='local',
    columns=['s', 'age'],
    total_rows=100  # More than stored
)
assert '(of 100 total)' in truncated_select.summary()
print(f"✓ Truncated SELECT handle: {truncated_select}")

# Test ASK result
ask_handle = SPARQLResultHandle(
    rows=True,
    result_type='ask',
    query='ASK { ?s ?p ?o }',
    endpoint='local'
)
assert ask_handle.summary() == "ASK: True"
print(f"✓ ASK handle: {ask_handle}")

# Test CONSTRUCT result
g = Graph()
g.add((URIRef('http://ex.org/alice'), URIRef('http://ex.org/age'), Literal('30')))
construct_handle = SPARQLResultHandle(
    rows=g,
    result_type='construct',
    query='CONSTRUCT { ?s ?p ?o } WHERE { ?s ?p ?o }',
    endpoint='local',
    triple_count=1,
    total_triples=1
)
assert construct_handle.summary() == "CONSTRUCT: 1 triples"
print(f"✓ CONSTRUCT handle: {construct_handle}")

# Test CONSTRUCT with truncation
truncated_construct = SPARQLResultHandle(
    rows=g,
    result_type='construct',
    query='CONSTRUCT { ?s ?p ?o } WHERE { ?s ?p ?o }',
    endpoint='local',
    triple_count=1,
    total_triples=500  # More than stored
)
assert '(of 500 total)' in truncated_construct.summary()
print(f"✓ Truncated CONSTRUCT handle: {truncated_construct}")

✓ SELECT handle: SPARQLResultHandle(SELECT: 1 rows, columns=['s', 'age'])
✓ Truncated SELECT handle: SPARQLResultHandle(SELECT: 1 rows (of 100 total), columns=['s', 'age'])
✓ ASK handle: SPARQLResultHandle(ASK: True)
✓ CONSTRUCT handle: SPARQLResultHandle(CONSTRUCT: 1 triples)
✓ Truncated CONSTRUCT handle: SPARQLResultHandle(CONSTRUCT: 1 triples (of 500 total))


<ipython-input-1-cbf62e56f2a5>:13: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp: str = field(default_factory=lambda: datetime.utcnow().isoformat() + 'Z')


## Remote SPARQL Query

Execute SPARQL queries against remote endpoints and return result handles.

## Query Rewriting Helper

Helper to inject LIMIT clauses into SELECT queries to bound server-side work.

In [ ]:
#| export
import re

def _inject_limit(query: str, limit: int) -> tuple[str, bool]:
    """Inject LIMIT clause into SELECT query if not already present.
    
    Args:
        query: SPARQL query string
        limit: LIMIT value to inject
        
    Returns:
        (modified_query, was_modified) tuple
    """
    query_upper = query.upper()
    
    # Only inject for SELECT queries
    if 'SELECT' not in query_upper:
        return query, False
    
    # Check if LIMIT already present
    if re.search(r'\bLIMIT\s+\d+', query_upper):
        return query, False
    
    # Find where to inject LIMIT (before ORDER BY if present, otherwise at end)
    order_match = re.search(r'\bORDER\s+BY\b', query_upper)
    if order_match:
        # Insert LIMIT before ORDER BY
        pos = order_match.start()
        modified = query[:pos] + f' LIMIT {limit} ' + query[pos:]
        return modified, True
    else:
        # Append LIMIT at end (before any trailing whitespace/comments)
        modified = query.rstrip() + f' LIMIT {limit}'
        return modified, True

In [ ]:
# Test LIMIT injection
q1 = "SELECT ?s ?p ?o WHERE { ?s ?p ?o }"
modified, injected = _inject_limit(q1, 100)
assert injected == True
assert 'LIMIT 100' in modified
print(f"✓ Basic injection: {modified}")

# Test with existing LIMIT (should not modify)
q2 = "SELECT ?s WHERE { ?s ?p ?o } LIMIT 50"
modified, injected = _inject_limit(q2, 100)
assert injected == False
assert modified == q2
print(f"✓ Existing LIMIT preserved: {modified}")

# Test with ORDER BY (inject before it)
q3 = "SELECT ?s ?o WHERE { ?s ?p ?o } ORDER BY ?s"
modified, injected = _inject_limit(q3, 100)
assert injected == True
assert 'LIMIT 100' in modified
assert modified.index('LIMIT') < modified.index('ORDER')
print(f"✓ Injection before ORDER BY: {modified}")

# Test CONSTRUCT (should not inject)
q4 = "CONSTRUCT { ?s ?p ?o } WHERE { ?s ?p ?o }"
modified, injected = _inject_limit(q4, 100)
assert injected == False
print(f"✓ CONSTRUCT not modified: {modified}")

✓ Basic injection: SELECT ?s ?p ?o WHERE { ?s ?p ?o } LIMIT 100
✓ Existing LIMIT preserved: SELECT ?s WHERE { ?s ?p ?o } LIMIT 50
✓ Injection before ORDER BY: SELECT ?s ?o WHERE { ?s ?p ?o }  LIMIT 100 ORDER BY ?s
✓ CONSTRUCT not modified: CONSTRUCT { ?s ?p ?o } WHERE { ?s ?p ?o }


In [ ]:
#| export
def sparql_query(
    query: str,
    endpoint: str = "https://query.wikidata.org/sparql",
    max_results: int = 100,
    name: str = 'res',
    ns: dict = None,
    timeout: float = 30.0,
    # Dataset integration
    ds_meta = None,
    store_in_work: bool = False,
    work_task_id: str = None
) -> str:
    """Execute SPARQL query, store SPARQLResultHandle in namespace.

    For SELECT: Stores SPARQLResultHandle with rows as list of dicts
    For CONSTRUCT/DESCRIBE: Stores SPARQLResultHandle with rdflib.Graph
    For ASK: Stores SPARQLResultHandle with boolean result

    IMPORTANT - Work Bounds:
    - For SELECT: Automatically injects LIMIT clause to bound server-side work
    - For CONSTRUCT/DESCRIBE: max_results only truncates locally; full results 
      still fetched from endpoint (SPARQL 1.1 has no standard LIMIT for graphs)

    If ds_meta provided and store_in_work=True:
    - CONSTRUCT results stored in work/<task_id> graph
    - Query logged to prov graph
    
    Args:
        query: SPARQL query string
        endpoint: SPARQL endpoint URL
        max_results: Maximum results to return (for SELECT/CONSTRUCT)
        name: Variable name to store result handle
        ns: Namespace dict (defaults to globals())
        timeout: Query timeout in seconds
        ds_meta: Optional DatasetMeta for dataset integration
        store_in_work: If True and ds_meta provided, store CONSTRUCT results in work graph
        work_task_id: Task ID for work graph (auto-generated if None)
        
    Returns:
        Summary string describing the result
    """
    if ns is None:
        ns = globals()
    
    # Try to inject LIMIT for SELECT queries to bound server-side work
    modified_query, limit_injected = _inject_limit(query, max_results)
    
    # Configure wrapper with timeout and headers
    headers = {"User-Agent": "RLM/1.0 (https://github.com/LA3D/rlm)"}
    wrapper = SPARQLWrapper(
        sparql_endpoint=endpoint,
        client_config=dict(timeout=timeout, headers=headers)
    )
    
    # Execute query with rdflib conversion (use modified_query if LIMIT was injected)
    result = wrapper.query(modified_query, convert=True)
    
    # Determine result type and create handle
    if isinstance(result, bool):
        # ASK query
        handle = SPARQLResultHandle(
            rows=result,
            result_type='ask',
            query=query,  # Store original query
            endpoint=endpoint
        )
        ns[name] = handle
        return f"ASK result: {result}, stored in '{name}'"
    
    elif hasattr(result, 'serialize'):
        # CONSTRUCT or DESCRIBE query - result is rdflib.Graph
        # NOTE: SPARQL 1.1 has no standard LIMIT for graph patterns, so we still
        # fetch full results from endpoint and truncate locally. This is a known
        # limitation - max_results is output truncation, not work bound.
        
        # Capture original count before truncation
        original_count = len(result)
        
        # Use islice to limit triples
        g = Graph()
        for triple in islice(result, max_results):
            g.add(triple)
        
        # Determine if CONSTRUCT or DESCRIBE
        query_upper = query.upper()
        result_type = 'construct' if 'CONSTRUCT' in query_upper else 'describe'
        
        handle = SPARQLResultHandle(
            rows=g,
            result_type=result_type,
            query=query,
            endpoint=endpoint,
            triple_count=len(g),        # Actual stored count
            total_triples=original_count  # Pre-truncation count
        )
        ns[name] = handle
        
        # Dataset integration for CONSTRUCT results
        if ds_meta is not None and store_in_work:
            from rlm.dataset import work_create
            import uuid
            
            task_id = work_task_id if work_task_id else f"sparql_{uuid.uuid4().hex[:8]}"
            graph_uri, work_graph = work_create(ds_meta, task_id)
            
            # Copy triples to work graph
            for s, p, o in g:
                work_graph.add((s, p, o))
            
            # Log query to prov
            from rdflib import Namespace, RDF, XSD
            RLM_PROV = Namespace('urn:rlm:prov:')
            event_uri = URIRef(f'urn:rlm:prov:sparql_{uuid.uuid4().hex[:8]}')
            ds_meta.prov.add((event_uri, RDF.type, RLM_PROV.SPARQLQuery))
            ds_meta.prov.add((event_uri, RLM_PROV.query, Literal(query)))
            ds_meta.prov.add((event_uri, RLM_PROV.endpoint, Literal(endpoint)))
            ds_meta.prov.add((event_uri, RLM_PROV.resultGraph, URIRef(graph_uri)))
            ds_meta.prov.add((event_uri, RLM_PROV.timestamp, Literal(handle.timestamp, datatype=XSD.dateTime)))
            ds_meta.prov.add((event_uri, RLM_PROV.session, Literal(ds_meta.session_id)))
            
            return f"Graph with {len(g)} triples stored in '{name}' and work/{task_id}" + \
                   (f" (of {original_count} total)" if original_count > len(g) else "")
        
        return f"Graph with {len(g)} triples stored in '{name}'" + \
               (f" (of {original_count} total)" if original_count > len(g) else "")
    
    else:
        # SELECT query - result is list of dicts
        # If we injected LIMIT, the result is already bounded
        result_list = list(result)
        original_count = len(result_list)
        
        # Apply additional client-side truncation if needed
        # (this should rarely happen if LIMIT injection worked)
        rows = list(islice(result_list, max_results))
        cols = list(rows[0].keys()) if rows else []
        
        handle = SPARQLResultHandle(
            rows=rows,
            result_type='select',
            query=query,  # Store original query
            endpoint=endpoint,
            columns=cols,
            total_rows=original_count  # This may equal len(rows) if LIMIT injection worked
        )
        ns[name] = handle
        
        msg = f"SELECT result with {len(rows)} rows, columns: {cols}, stored in '{name}'"
        if limit_injected:
            msg += f" (LIMIT {max_results} injected)"
        return msg

Test against Wikidata:

In [ ]:
#| eval: false
# Test SELECT query against Wikidata
test_ns = {}
result = sparql_query(
    "SELECT ?s ?p ?o WHERE { ?s ?p ?o } LIMIT 5",
    ns=test_ns,
    name='wikidata_test'
)
print(result)
assert 'wikidata_test' in test_ns
assert isinstance(test_ns['wikidata_test'], SPARQLResultHandle)
assert test_ns['wikidata_test'].result_type == 'select'
assert len(test_ns['wikidata_test'].rows) == 5
print(f"✓ SELECT query works: {test_ns['wikidata_test'].summary()}")

# Test CONSTRUCT query
result = sparql_query(
    "CONSTRUCT { ?s ?p ?o } WHERE { ?s ?p ?o } LIMIT 3",
    ns=test_ns,
    name='graph_test'
)
print(result)
assert test_ns['graph_test'].result_type == 'construct'
assert isinstance(test_ns['graph_test'].rows, Graph)
print(f"✓ CONSTRUCT query works: {test_ns['graph_test'].summary()}")

SELECT result with 5 rows, columns: ['s', 'p', 'o'], stored in 'wikidata_test'
✓ SELECT query works: SELECT: 5 rows, columns=['s', 'p', 'o']
Graph with 3 triples stored in 'graph_test'
✓ CONSTRUCT query works: CONSTRUCT: 3 triples


<ipython-input-1-cbf62e56f2a5>:13: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp: str = field(default_factory=lambda: datetime.utcnow().isoformat() + 'Z')


## Local Graph Query

Execute SPARQL queries against local rdflib graphs (mounted ontologies or work graphs).

In [ ]:
#| export
def sparql_local(
    query: str,
    graph: Graph | str,
    max_results: int = 100,
    name: str = 'res',
    ns: dict = None
) -> str:
    """Execute SPARQL query on local rdflib Graph.

    Useful for querying mounted ontologies or work graphs.
    Returns SPARQLResultHandle same as sparql_query().
    
    IMPORTANT - Work Bounds:
    - max_results is output truncation only; full result set is materialized
    - For large local graphs, consider filtering in the SPARQL query itself
    
    Args:
        query: SPARQL query string
        graph: rdflib.Graph object or name of graph in namespace
        max_results: Maximum results to return
        name: Variable name to store result handle
        ns: Namespace dict (defaults to globals())
        
    Returns:
        Summary string describing the result
    """
    if ns is None:
        ns = globals()
    
    # Resolve graph if string name provided
    if isinstance(graph, str):
        if graph not in ns:
            return f"Error: Graph '{graph}' not found in namespace"
        graph_obj = ns[graph]
        # Handle GraphMeta wrapper
        if hasattr(graph_obj, 'graph'):
            graph_obj = graph_obj.graph
    else:
        graph_obj = graph
    
    if not isinstance(graph_obj, Graph):
        return f"Error: Expected rdflib.Graph, got {type(graph_obj)}"
    
    # Execute query on local graph
    result = graph_obj.query(query)
    
    # Determine result type
    query_upper = query.upper()
    
    if 'ASK' in query_upper:
        # ASK query
        ask_result = bool(result)
        handle = SPARQLResultHandle(
            rows=ask_result,
            result_type='ask',
            query=query,
            endpoint='local'
        )
        ns[name] = handle
        return f"ASK result: {ask_result}, stored in '{name}'"
    
    elif 'CONSTRUCT' in query_upper or 'DESCRIBE' in query_upper:
        # CONSTRUCT or DESCRIBE query
        result_type = 'construct' if 'CONSTRUCT' in query_upper else 'describe'
        
        # Capture original count before truncation
        result_list = list(result)
        original_count = len(result_list)
        
        # Use islice to limit triples
        g = Graph()
        for triple in islice(result_list, max_results):
            g.add(triple)
        
        handle = SPARQLResultHandle(
            rows=g,
            result_type=result_type,
            query=query,
            endpoint='local',
            triple_count=len(g),        # Actual stored count
            total_triples=original_count  # Pre-truncation count
        )
        ns[name] = handle
        
        msg = f"Graph with {len(g)} triples stored in '{name}'"
        if original_count > len(g):
            msg += f" (of {original_count} total)"
        return msg
    
    else:
        # SELECT query
        # Capture original count before truncation
        result_list = list(result)
        original_count = len(result_list)
        
        rows = []
        for row in islice(result_list, max_results):
            row_dict = {}
            for var in result.vars:
                row_dict[str(var)] = row[var] if row[var] else None
            rows.append(row_dict)
        
        cols = [str(v) for v in result.vars] if result.vars else []
        
        handle = SPARQLResultHandle(
            rows=rows,
            result_type='select',
            query=query,
            endpoint='local',
            columns=cols,
            total_rows=original_count  # Pre-truncation count
        )
        ns[name] = handle
        
        msg = f"SELECT result with {len(rows)} rows, columns: {cols}, stored in '{name}'"
        if original_count > len(rows):
            msg += f" (of {original_count} total)"
        return msg

Test with local graph:

In [ ]:
# Create test graph
test_graph = Graph()
test_graph.add((URIRef('http://ex.org/alice'), URIRef('http://ex.org/age'), Literal('30')))
test_graph.add((URIRef('http://ex.org/bob'), URIRef('http://ex.org/age'), Literal('25')))
test_graph.add((URIRef('http://ex.org/alice'), URIRef('http://ex.org/city'), Literal('Boston')))

test_ns = {'my_graph': test_graph}

# Test SELECT query on local graph
result = sparql_local(
    "SELECT ?s ?age WHERE { ?s <http://ex.org/age> ?age }",
    'my_graph',
    ns=test_ns,
    name='local_res'
)
print(result)
assert 'local_res' in test_ns
assert test_ns['local_res'].result_type == 'select'
assert len(test_ns['local_res'].rows) == 2
assert test_ns['local_res'].total_rows == 2
print(f"✓ Local SELECT query works: {test_ns['local_res'].rows}")

# Test CONSTRUCT on local graph
result = sparql_local(
    "CONSTRUCT { ?s <http://ex.org/age> ?age } WHERE { ?s <http://ex.org/age> ?age }",
    test_graph,
    ns=test_ns,
    name='local_graph'
)
print(result)
assert test_ns['local_graph'].result_type == 'construct'
assert len(test_ns['local_graph'].rows) == 2
assert test_ns['local_graph'].triple_count == 2
assert test_ns['local_graph'].total_triples == 2
print(f"✓ Local CONSTRUCT query works")

# Test truncation
result = sparql_local(
    "CONSTRUCT { ?s ?p ?o } WHERE { ?s ?p ?o }",
    test_graph,
    max_results=2,
    ns=test_ns,
    name='truncated'
)
print(result)
assert len(test_ns['truncated'].rows) == 2
assert test_ns['truncated'].triple_count == 2
assert test_ns['truncated'].total_triples == 3  # Original had 3
assert '(of 3 total)' in test_ns['truncated'].summary()
print(f"✓ Truncation works correctly: {test_ns['truncated'].summary()}")

SELECT result with 2 rows, columns: ['s', 'age'], stored in 'local_res'
✓ Local SELECT query works: [{'s': rdflib.term.URIRef('http://ex.org/alice'), 'age': rdflib.term.Literal('30')}, {'s': rdflib.term.URIRef('http://ex.org/bob'), 'age': rdflib.term.Literal('25')}]
Graph with 2 triples stored in 'local_graph'
✓ Local CONSTRUCT query works
Graph with 2 triples stored in 'truncated' (of 3 total)
✓ Truncation works correctly: CONSTRUCT: 2 triples (of 3 total)


<ipython-input-1-cbf62e56f2a5>:13: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp: str = field(default_factory=lambda: datetime.utcnow().isoformat() + 'Z')


## View Operations

Bounded view functions for progressive disclosure over result sets.

These functions work with `SPARQLResultHandle`, `ResultTable`, or plain lists.

In [ ]:
#| export
def res_sample(result, n: int = 10, seed: int = None) -> list:
    """Get random sample of N rows from result.

    Args:
        result: SPARQLResultHandle, ResultTable, or list
        n: Number of rows to sample
        seed: Optional random seed for reproducibility

    Returns:
        List of sampled rows
    """
    if seed is not None:
        random.seed(seed)

    # Extract rows from different result types
    if isinstance(result, SPARQLResultHandle):
        if result.result_type in ['construct', 'describe']:
            # For graphs, sample triples
            rows = list(result.rows)
        elif result.result_type == 'ask':
            # ASK has no rows to sample
            return [result.rows]
        else:
            rows = result.rows
    elif hasattr(result, 'rows'):
        # ResultTable or similar
        rows = result.rows
    else:
        # Plain list
        rows = result

    if len(rows) <= n:
        return list(rows)
    return random.sample(list(rows), n)

Test res_sample:

In [ ]:
# Test with list
test_list = [{'x': i} for i in range(20)]
sample = res_sample(test_list, n=5, seed=42)
assert len(sample) == 5
assert all(isinstance(item, dict) for item in sample)
print(f"✓ res_sample works with list: {len(sample)} items")

# Test with SPARQLResultHandle
handle = SPARQLResultHandle(
    rows=[{'s': f'http://ex.org/item{i}'} for i in range(15)],
    result_type='select',
    query='SELECT ?s WHERE { ?s ?p ?o }',
    endpoint='local',
    columns=['s'],
    total_rows=15
)
sample = res_sample(handle, n=3, seed=42)
assert len(sample) == 3
print(f"✓ res_sample works with SPARQLResultHandle")

# Test with small result (no sampling needed)
small_list = [1, 2, 3]
sample = res_sample(small_list, n=10)
assert len(sample) == 3
print(f"✓ res_sample handles small results correctly")

✓ res_sample works with list: 5 items
✓ res_sample works with SPARQLResultHandle
✓ res_sample handles small results correctly


<ipython-input-1-cbf62e56f2a5>:13: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp: str = field(default_factory=lambda: datetime.utcnow().isoformat() + 'Z')


## Setup Function

Initialize SPARQL tools in namespace for RLM sessions.

In [ ]:
#| export
def setup_sparql_context(
    ns: dict,
    default_endpoint: str = "https://query.wikidata.org/sparql",
    ds_meta = None
) -> str:
    """Initialize SPARQL tools in namespace.

    Binds:
    - sparql_query() with default endpoint
    - sparql_local() if ds_meta provided
    - res_head(), res_where(), res_group(), res_distinct(), res_sample()

    Args:
        ns: Namespace dict where functions will be bound
        default_endpoint: Default SPARQL endpoint URL
        ds_meta: Optional DatasetMeta for dataset integration
        
    Returns:
        Status message
    """
    # Import view functions from dataset module
    try:
        from rlm.dataset import res_head, res_where, res_group, res_distinct
    except ImportError:
        # Fallback if dataset module not available
        res_head = res_where = res_group = res_distinct = None
    
    # Bind sparql_query with default endpoint and dataset integration
    if ds_meta is not None:
        ns['sparql_query'] = partial(sparql_query, endpoint=default_endpoint, ns=ns, ds_meta=ds_meta)
    else:
        ns['sparql_query'] = partial(sparql_query, endpoint=default_endpoint, ns=ns)
    
    # Bind sparql_local
    ns['sparql_local'] = partial(sparql_local, ns=ns)
    
    # Bind view operations
    if res_head is not None:
        ns['res_head'] = res_head
        ns['res_where'] = res_where
        ns['res_group'] = res_group
        ns['res_distinct'] = res_distinct
    ns['res_sample'] = res_sample
    
    bound_funcs = ['sparql_query', 'sparql_local', 'res_sample']
    if res_head is not None:
        bound_funcs.extend(['res_head', 'res_where', 'res_group', 'res_distinct'])
    
    msg = f"SPARQL context initialized with endpoint: {default_endpoint}"
    if ds_meta is not None:
        msg += f"\nDataset integration enabled (session: {ds_meta.session_id})"
    msg += f"\nBound functions: {', '.join(bound_funcs)}"
    
    return msg

Test setup function:

In [ ]:
# Test basic setup
test_ns = {}
result = setup_sparql_context(test_ns)
print(result)
assert 'sparql_query' in test_ns
assert 'sparql_local' in test_ns
assert 'res_sample' in test_ns
print(f"✓ Setup function works")

# Test with dataset integration
try:
    from rlm.dataset import DatasetMeta
    from rdflib import Dataset
    
    ds = Dataset()
    ds_meta = DatasetMeta(ds, name='test')
    
    test_ns2 = {}
    result = setup_sparql_context(test_ns2, ds_meta=ds_meta)
    print(result)
    assert 'session:' in result
    print(f"✓ Setup with dataset integration works")
except ImportError:
    print("⊘ Dataset module not available, skipping integration test")

SPARQL context initialized with endpoint: https://query.wikidata.org/sparql
Bound functions: sparql_query, sparql_local, res_sample, res_head, res_where, res_group, res_distinct
✓ Setup function works
SPARQL context initialized with endpoint: https://query.wikidata.org/sparql
Dataset integration enabled (session: ee270120)
Bound functions: sparql_query, sparql_local, res_sample, res_head, res_where, res_group, res_distinct
✓ Setup with dataset integration works


## Usage Examples

End-to-end examples showing SPARQL handles in RLM context.

In [ ]:
#| eval: false
# Example 1: Basic SPARQL workflow
ns = {}
setup_sparql_context(ns)

# Execute query (LLM would do this)
ns['sparql_query']('SELECT ?s ?p ?o WHERE { ?s ?p ?o } LIMIT 10', name='results')

# Inspect results
print(ns['results'].summary())
print(res_head(ns['results'], 5))
print(res_sample(ns['results'], 3))

SELECT: 10 rows, columns=['s', 'p', 'o']
---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
Cell In[1], line 11
      9 # Inspect results
     10 print(ns['results'].summary())
---> 11 print(res_head(ns['results'], 5))
     12 print(res_sample(ns['results'], 3))

NameError: name 'res_head' is not defined


<ipython-input-1-cbf62e56f2a5>:13: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp: str = field(default_factory=lambda: datetime.utcnow().isoformat() + 'Z')


NameError: name 'res_head' is not defined

In [ ]:
#| eval: false
# Example 2: Dataset integration
from rlm.dataset import setup_dataset_context

ns = {}
setup_dataset_context(ns)
setup_sparql_context(ns, ds_meta=ns['ds_meta'])

# Query and store in work graph
ns['sparql_query'](
    'CONSTRUCT { ?s ?p ?o } WHERE { ?s ?p ?o } LIMIT 5',
    name='discovered_triples',
    store_in_work=True,
    work_task_id='discovery_1'
)

# Check provenance
print(ns['dataset_stats']())

Dataset 'ds' (session: 2fbcf046)
mem: 0 triples
prov: 6 events
work graphs: 1
onto graphs: 0


<ipython-input-1-cbf62e56f2a5>:13: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp: str = field(default_factory=lambda: datetime.utcnow().isoformat() + 'Z')


In [ ]:
#| eval: false
# Example 3: Local graph queries
from rlm.ontology import setup_ontology_context

ns = {}
setup_sparql_context(ns)
setup_ontology_context('ontology/prov.ttl', ns, name='prov')

# Query mounted ontology
ns['sparql_local'](
    'SELECT ?c WHERE { ?c a <http://www.w3.org/2002/07/owl#Class> }',
    'prov',
    name='classes'
)

print(f"Found {len(ns['classes'].rows)} classes")
print(res_head(ns['classes'], 10))

Found 59 classes
---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
Cell In[1], line 17
     10 ns['sparql_local'](
     11     'SELECT ?c WHERE { ?c a <http://www.w3.org/2002/07/owl#Class> }',
     12     'prov',
     13     name='classes'
     14 )
     16 print(f"Found {len(ns['classes'].rows)} classes")
---> 17 print(res_head(ns['classes'], 10))

NameError: name 'res_head' is not defined


<ipython-input-1-cbf62e56f2a5>:13: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp: str = field(default_factory=lambda: datetime.utcnow().isoformat() + 'Z')


NameError: name 'res_head' is not defined